In [2]:
import pandas as pd
df = pd.read_csv("Doc\\ipodata.csv")
df.head()

,Code,Name,Date,Price,Share,Amount,Warran,Underwriter,Sector,Subsector,Field,Website,Prospectus,Summary_Prospectus,Alamat,Image,Papan Pencatatan
0,GUNA,PT. Gunanusa Eramandiri Tbk.,09 Jul 2024,Rp150,20%,"5,000,000 lot",0,GR - PANIN SEKURITAS TBK.,Barang Konsumen Primer,Makanan & Minuman,Industri makanan dari kedele dan kacang-kacang...,https://www.gunanusaeramandiri.com,https://storage.googleapis.com/assets.trading....,https://storage.googleapis.com/assets.trading....,Kawasan Industri BIIE Hyundai Jl. Inti I Kav. ...,https://storage.googleapis.com/assets.trading....,Pengembangan
1,GOLF,PT Intra GolfLink Resorts Tbk,08 Jul 2024,Rp200,15.02%,"31,000,000 lot",0,IF - SAMUEL SEKURITAS INDONESIA,Barang Konsumen Non-Primer,Barang Rekreasi,Bergerak Dalam Bidang Jasa Konsultasi Manajeme...,https://www.golflinkresorts.co.id/id,https://storage.googleapis.com/assets.trading....,https://storage.googleapis.com/assets.trading....,"Desa Kadumanggu, Kecamatan Madang Bogor 16810 ...",https://storage.googleapis.com/assets.trading....,Utama
2,LABS,PT UBC Medical Indonesia Tbk,10 Jul 2024,Rp102,17.72%,"7,000,000 lot",0,YJ - LOTUS ANDALAN SEKURITAS,Kesehatan,Jasa & Peralatan Kesehatan,Distributor Alat Kesehatan,https://www.ubcindonesia.com,https://storage.googleapis.com/assets.trading....,https://storage.googleapis.com/assets.trading....,"Kawasan Industri Pulogadung, Jl. Rawagelam V, ...",https://storage.googleapis.com/assets.trading....,Pengembangan
3,ISEA,PT Indo American Seafoods Tbk,08 Jul 2024,Rp250,20.86%,"2,900,000 lot",2 : 1,CP - VALBURY SEKURITAS INDONESIA,Barang Konsumen Primer,Makanan & Minuman,"Industri Pembekuan Biota Air Lainnya, Industri...",https://www.indoamericanseafoods.com/,https://storage.googleapis.com/assets.trading....,https://storage.googleapis.com/assets.trading....,"Jl. Ir. Sutami KM 13, Desa Sukanegara, Kecamat...",https://storage.googleapis.com/assets.trading....,Pengembangan
4,PART,PT Cipta Perdana Lancar,05 Jul 2024,Rp105,25%,"6,800,000 lot",1 : 1,EP - MNC SEKURITAS,Barang Konsumen Non-Primer,Otomotif & Komponen Otomotif,Perseroan bergerak di bidang Manufaktur Kompon...,https://www.ciptaperdanalancar.com,https://storage.googleapis.com/assets.trading....,https://storage.googleapis.com/assets.trading....,"Jl. Prabu Siliwangi KM 0,5 Keroncong, Kec. Jat...",https://storage.googleapis.com/assets.trading....,Pengembangan


In [3]:
df["Price"] = df["Price"].str.replace("Rp","").astype(int)
prices = df["Price"]
df["Share"] = df["Share"].str.replace("%","").astype(float)
df["Warran"] = df['Warran'].apply(lambda x: int(x.split(":")[0])/int(x.split(":")[1]) if x != "0" else int(x) )
df["Underwriter"] = df["Underwriter"].apply(lambda x:x.split("-")[0])
df["Amount"] = df["Amount"].str.replace(",","")
df["Amount"] = df["Amount"].str.replace("lot","")
df["Amount"] = df["Amount"].apply(lambda x:int(x))

In [4]:
from sklearn.preprocessing import LabelEncoder
# Initialize the LabelEncoder
encoder = LabelEncoder()
# Encode each column
for column in ["Papan Pencatatan","Underwriter","Sector","Subsector"]:
    df[column] = encoder.fit_transform(df[column])

In [5]:
feature_enginering = ["Papan Pencatatan","Share","Underwriter","Warran","Sector","Subsector","Price","Amount"]

In [6]:
x = df[feature_enginering]

In [8]:
target = pd.read_csv("Doc//ipo_gain.csv")
target.head()

,1 candle gain,2 candle gain,3 candle gain,4 candle gain,5 candle gain,1 candle high,2 candle high,3 candle high,4 candle high,5 candle high
0,34.666667,68.000000,109.333333,98.666667,94.666667,34.666667,68.000000,109.333333,161.333333,116.000000
1,35.000000,36.000000,20.000000,20.000000,15.000000,35.000000,51.000000,38.000000,31.000000,21.000000
2,34.313725,80.392157,92.156863,66.666667,76.470588,34.313725,80.392157,123.529412,109.803922,88.235294
3,24.800000,28.000000,52.000000,87.200000,40.800000,24.800000,40.000000,52.000000,88.000000,98.400000
4,24.761905,4.761905,10.476190,16.190476,22.857143,34.285714,28.571429,19.047619,18.095238,22.857143


In [22]:
for label in target.columns:
    print(f"Label   :{label}")
    y = target[label]
    x = df[feature_enginering]
    attemp = label

    import numpy as np
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    from sklearn.linear_model import LinearRegression, Ridge, Lasso
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.svm import SVR
    from xgboost import XGBRegressor
    import joblib
    X = x
    y = y

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the models and their hyperparameters for tuning
    models = {
        'LinearRegression': (LinearRegression(), {}),
        'Ridge': (Ridge(), {'alpha': [0.1, 1, 10]}),
        'Lasso': (Lasso(), {'alpha': [0.1, 1, 10]}),
        'DecisionTree': (DecisionTreeRegressor(), {'max_depth': [3, 5, 10]}),
        'RandomForest': (RandomForestRegressor(), {'n_estimators': [50, 100], 'max_depth': [3, 5, 10]}),
        'SVR': (SVR(), {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']}),
        'XGBRegressor': (XGBRegressor(), {'learning_rate': [0.01, 0.1], 'max_depth': [3, 5], 'n_estimators': [50, 100]})
    }

    from sklearn.metrics import make_scorer
    # Define the custom error function
    def mean_squared_negative_error(y_true, y_pred):
        error = np.where(y_true < y_pred, (y_true - y_pred)**2, 0)
        return np.mean(error)

    def mean_absolute_negative_error(y_true, y_pred):
        error = np.where(y_true < y_pred, np.abs(y_true - y_pred), 0)
        return np.mean(error)

    def custom_r2_score(y_true, y_pred):
        ss_res = np.sum(np.where(y_true < y_pred, (y_true - y_pred) ** 2, 0))
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
        return 1 - ss_res / ss_tot if ss_tot != 0 else 0

    # Create scorers using make_scorer
    custom_scorer_msne = make_scorer(mean_squared_negative_error, greater_is_better=False)
    custom_scorer_mane = make_scorer(mean_absolute_negative_error, greater_is_better=False)

    # Initialize a list to store the results
    best_mane = float('inf')
    best_model = None
    best_model_name = ""
    results_list = []
    # Use the custom scorer in GridSearchCV
    for model_name, (model, params) in models.items():
        grid_search = GridSearchCV(model, params, cv=5, scoring=custom_scorer_msne)
        grid_search.fit(X_train, y_train)
        
        best_model_instance = grid_search.best_estimator_
        y_pred = best_model_instance.predict(X_test)
        
        mane = mean_absolute_negative_error(y_test, y_pred)
        msne = mean_squared_negative_error(y_test, y_pred)
        rmsne = np.sqrt(msne)
        custom_r2 = custom_r2_score(y_test, y_pred)
        
        results_list.append({
            'Model': model_name,
            'BestParams': grid_search.best_params_,
            'MANE': mane,
            'MSNE': msne,
            'RMSNE': rmsne,
            'R²': custom_r2
        })
        # Save the best model based on MANE
        if mane < best_mane:
            best_mane = mane
            best_model = best_model_instance
            best_model_name = model_name

    # Convert results list to a DataFrame
    results = pd.DataFrame(results_list)
    # Save all model results to a CSV file
    results.to_csv(f'Doc/model_results_{attemp}.csv', index=False)
    # Save the best model to a file
    joblib.dump(best_model, f'Doc/best_model_{attemp}.pkl')
    print(f"All model results saved to 'Doc/model_results_{attemp}.csv'.")
    print(f"Best model ({best_model_name}) saved to 'Doc/best_model_{attemp}.pkl' with MAE: {best_mane}")

    import pandas as pd
    import joblib
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    # Load the best model from the file
    best_model = joblib.load(f'Doc/best_model_{attemp}.pkl')
    X = x
    y = y
    # Make predictions using the best model
    y_pred = best_model.predict(X)
    # Calculate the errors for each row
    errors = pd.DataFrame({
        'Actual': y,
        'Predicted': y_pred,
        'Error': y - y_pred,
        'NegativeError': np.where(y < y_pred, np.abs(y - y_pred), 0),
        'SquaredNError': np.where(y < y_pred, (y - y_pred) ** 2, 0)
    })

    # Show the errors for each row
    # print(errors)

    # Calculate and print overall metrics
    mane = mean_absolute_negative_error(y, y_pred)
    msne = mean_squared_negative_error(y, y_pred)
    rmsne = np.sqrt(msne)
    custom_r2 = custom_r2_score(y, y_pred)

    print(f"Overall MANE: {mane}")
    print(f"Overall MSNE: {msne}")
    print(f"Overall RMSNE: {rmsne}")
    print(f"Overall R²: {custom_r2}")

    errors.insert(0, 'Code', df['Code'])
    errors.to_csv(f"Doc/error{attemp}.csv",index=False)
    print("\n")

Label   :1 candle gain


C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=1.60394e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=3.50034e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.22964e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.01454e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True

All model results saved to 'Doc/model_results_1 candle gain.csv'.
Best model (RandomForest) saved to 'Doc/best_model_1 candle gain.pkl' with MAE: 6.335196453170472
Overall MANE: 3.2273753761267003
Overall MSNE: 47.22522432564584
Overall RMSNE: 6.872061141000263
Overall R²: 0.842127029564979


Label   :2 candle gain


C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=1.60394e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=3.50034e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.22964e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.01454e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True

All model results saved to 'Doc/model_results_2 candle gain.csv'.
Best model (SVR) saved to 'Doc/best_model_2 candle gain.pkl' with MAE: 12.735164062749982
Overall MANE: 7.326227946537351
Overall MSNE: 193.15368846675364
Overall RMSNE: 13.897974257666245
Overall R²: 0.8294319987799643


Label   :3 candle gain


...

All model results saved to 'Doc/model_results_3 candle high.csv'.
Best model (SVR) saved to 'Doc/best_model_3 candle high.pkl' with MAE: 15.56536546088948
Overall MANE: 12.196977635424666
Overall MSNE: 415.62286063608525
Overall RMSNE: 20.38683056868049
Overall R²: 0.8376569936449298


Label   :4 candle high


C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=1.60394e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=3.50034e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.22964e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.01454e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True

All model results saved to 'Doc/model_results_4 candle high.csv'.
Best model (SVR) saved to 'Doc/best_model_4 candle high.pkl' with MAE: 15.69950367496939
Overall MANE: 12.525101267906672
Overall MSNE: 455.82678814367074
Overall RMSNE: 21.350100424674135
Overall R²: 0.8845164012323234


Label   :5 candle high


C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=1.60394e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=3.50034e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.22964e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_ridge.py:211: LinAlgWarning: Ill-conditioned matrix (rcond=2.01454e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True

All model results saved to 'Doc/model_results_5 candle high.csv'.
Best model (SVR) saved to 'Doc/best_model_5 candle high.pkl' with MAE: 16.326651597854667
Overall MANE: 13.014917094467853
Overall MSNE: 492.3340496642474
Overall RMSNE: 22.188601796062937
Overall R²: 0.9085480343658


